In [55]:
import re
from collections import defaultdict, Counter
from pprint import pp
import json
dumps = lambda thing: print(json.dumps(thing, indent=4))

POINTS = 12
sample = open('d19.sample').read()

scanner_find = re.compile(r'--- scanner (\d+) ---\s+(.*?)(?=\Z|\s{2,})', flags=re.S|re.M).findall
coord_find = re.compile(r'(-?\d+),(-?\d+),(-?\d+)').findall



In [78]:
def one_n_rest(iterable):

    # one_n_rest("ABC") -> (A, BC), (B, AC), (C, AB)
    for one_i, one in enumerate(iterable):
        yield one, (rest for rest_i, rest in enumerate(iterable) if rest_i != one_i)



test = one_n_rest("ABC")

for one, rest in test:
    for thing in rest:
        print (one, thing)
    print ("")



A B
A C

B A
B C

C A
C B



In [79]:

# look down at z and turn right
# look at x to z and turn right
# look at y to z and turn right

"""
123
rot z
rot z
rot z

rot x
452

rot z
rot z
rot z

rot x
536

rot z
rot z
rot z

rot y
rot y
214

rot z
rot z
rot z

rot x
351

rot z
rot z
rot z

rot x
645

rot z
rot z
rot z


"""

def rx(coord):
    x,y,z = coord
    return x,-z,y

def w1(coord):
    x,y,z = coord
    return -x,-z,-y

def w2(coord):
    x,y,z = coord
    return x,z,-y


def ry2(coord):
    x,y,z = coord
    return -x,y,-z

def rz(coord):
    x,y,z = coord
    return -y,x,z

rotations = [rz,rz,rz,rx,rz,rz,rz,rx,rz,rz,rz,ry2,rz,rz,rz,rx,rz,rz,rz,rx,rz,rz,rz]

def get_rotations(coord):
    yield coord
    for rot in rotations:
        coord = rot(coord)
        yield coord



In [ ]:


import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


#5
numbers = {
5:[  
(1,0,1),
(3,0,1),
(2,0,2),
(1,0,3),
(3,0,3),],
1:[
(0,2,2),],
4:[
(1,1,4),
(3,1,4),
(1,3,4),
(3,3,4),],
3:[
(1,1,0),
(2,2,0),
(3,3,0),],
2:[
(3,4,1),
(1,4,3),
],
6:[  
(4,1,1),
(4,1,2),
(4,1,3),
(4,3,1),
(4,3,2),
(4,3,3),],
}

colors = {5: 'b', 1: 'g', 2: 'y', 3: 'r', 4: 'm', 6: 'c'}

numiters = {face: [get_rotations(o) for o in orientations] for face, orientations in numbers.items()}

for i in range(23):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    for group, coords in numiters.items():
        for coord in coords:   
            x,y,z = next(coord)
            ax.scatter([x/2],[y/2], z/2,color=colors[group])

In [107]:
import time
TIMER = False
minimum = (12 * (12 - 1)) // 2

DEBUG = False
def get_scanners_and_coords(data):
    results = defaultdict(list)
    for scanner_ids, coords in scanner_find(data):
        scanner_id = int(scanner_ids)
        for x,y,z in coord_find(coords):
            results[scanner_id].append((int(x),int(y),int(z)))
    return dict(results)

def get_distance(coord1, coord2):
    x1,y1,z1 = coord1
    x2,y2,z2 = coord2
    return ((x2 - x1) ** 2) + ((y2 - y1) ** 2) + ((z2 - z1) ** 2)

def get_3d_distances(data):
    start = time.perf_counter()
    results = defaultdict(list)
    for scanner, coords in data.items():
        for me, remaining in one_n_rest(coords):
                for theirs in remaining:
                    distance = get_distance(me, theirs)
                    DEBUG and print(f'{me=} {theirs=} {distance=}')
                    results[scanner].append(distance)
    TIMER and print(f'get_3d_distances: {(time.perf_counter() - start)=}')
    return results


def get_3d_distance_pairs(threed_distances):
    start = time.perf_counter()
    completed = []
    results = []
    for scanner, others in one_n_rest(threed_distances):
        for other in others:
            # DEBUG and print (f'{scanner=} {other=}')
            id = tuple(sorted([scanner, other]))
            compare1 = list(set(threed_distances[scanner])) 
            compare2 = list(set(threed_distances[other]))
            counts = Counter(compare1 + compare2)
            commons = len([c for c in counts.values() if c > 1])
            if commons >= minimum:
                results.append(id)
    TIMER and print(f'get_3d_distance_pairs: {(time.perf_counter() - start)=}')
    return tuple(set(results))

                


In [103]:
def do_whatever(data):

    scanners_and_coords = get_scanners_and_coords(data)
    threed_distances = get_3d_distances(scanners_and_coords)

    pairs = get_3d_distance_pairs(threed_distances)

    results = []

    def get_diff(coord1, coord2):
        x1,y1,z1 = coord1
        x2,y2,z2 = coord2
        return x1 - x2, y1 - y2, z1 - z2


    def add_diff(coord, diff):
        x1,y1,z1 = coord
        x2,y2,z2 = diff
        return x1 + x2, y1 + y2, z1 + z2



    result = {k: len(v) for k,v in scanners_and_coords.items()}
    new_result = None
    broken = 0
    while True:
        if new_result == result:
            break
        result = new_result
        threed_distances = get_3d_distances(scanners_and_coords)
        pairs = get_3d_distance_pairs(threed_distances)
        for me, them in pairs:
            # DEBUG and print(f'{me=} {them=}')
            for theirs in zip(*map(get_rotations, scanners_and_coords[them])):
                diffs = defaultdict(int)
                mine = scanners_and_coords[me]
                for mcoord in mine:
                    for tcoord in theirs:
                        diffs[get_diff(mcoord,tcoord)]+=1
                good = [k for k,v in diffs.items() if v >= 12]
                # if not good and (plural := {k:v for k,v in diffs.items() if v > 1}):
                #     pp(plural)
                if good:
                    # print ("good")
                    diff = good[0]
                    for coord in theirs:
                        added = add_diff(coord, diff)
                        scanners_and_coords[me].append(added)
                    scanners_and_coords[me] = list(set(scanners_and_coords[me]))
                    break
        new_result = {k: len(v) for k,v in scanners_and_coords.items()}
        # print (new_result)
    return result
            





            

In [100]:
do_whatever(sample)

{0: 79, 1: 66, 2: 40, 3: 25, 4: 26}

In [108]:
TIMER = True
DEBUG = False
do_whatever(open('d19.input').read()) 

#distance fingerprinting sucks.  come back to this

get_3d_distances: (time.perf_counter() - start)=0.00531183999919449
get_3d_distance_pairs: (time.perf_counter() - start)=0.0872631909987831
get_3d_distances: (time.perf_counter() - start)=0.005705452000256628
get_3d_distance_pairs: (time.perf_counter() - start)=0.06432149300053425
get_3d_distances: (time.perf_counter() - start)=0.018769901000268874
get_3d_distance_pairs: (time.perf_counter() - start)=0.24798858599933737
get_3d_distances: (time.perf_counter() - start)=0.1546432979994279
get_3d_distance_pairs: (time.perf_counter() - start)=2.580143467999733
get_3d_distances: (time.perf_counter() - start)=0.25677599600021495
get_3d_distance_pairs: (time.perf_counter() - start)=4.545808754000973
